In [122]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [123]:
DATA_PATH = '../data/'

users = pd.read_csv(DATA_PATH + 'users.csv')
books = pd.read_csv(DATA_PATH + 'books.csv')
train = pd.read_csv(DATA_PATH + 'train_ratings.csv')
test = pd.read_csv(DATA_PATH + 'test_ratings.csv')
sub = pd.read_csv(DATA_PATH + 'sample_submission.csv')

In [134]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149570 entries, 0 to 149569
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 149570 non-null  object 
 1   book_title           149570 non-null  object 
 2   book_author          149570 non-null  object 
 3   year_of_publication  149570 non-null  float64
 4   publisher            149570 non-null  object 
 5   img_url              149570 non-null  object 
 6   language             82343 non-null   object 
 7   category             80719 non-null   object 
 8   summary              82343 non-null   object 
 9   img_path             149570 non-null  object 
dtypes: float64(1), object(9)
memory usage: 11.4+ MB


In [124]:
users['location_city'] = users['location'].apply(lambda x: x.split(',')[0])
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1])
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2])
users = users.drop(['location'], axis=1)

In [125]:
len(users['location_country'].unique())

347

In [126]:
users['location_country'].value_counts()[:30]

 usa               43991
 canada             6208
 germany            3526
 united kingdom     2914
                    2101
 australia          1791
 spain              1653
 france              794
 italy               770
 switzerland         451
 new zealand         448
 netherlands         403
 portugal            325
 austria             251
 malaysia            167
 singapore           129
 sweden              120
 ireland             107
 belgium             104
 brazil              103
 finland              89
 philippines          80
 argentina            77
 japan                74
 mexico               65
 denmark              61
 norway               54
 israel               48
 india                39
 romania              39
Name: location_country, dtype: int64

In [127]:
users['location_city'] = users['location_city'].str.strip()
users['location_state'] = users['location_state'].str.strip()
users['location_country'] = users['location_country'].str.strip()

In [128]:
users['location_city'] = users['location_city'].str.replace(r'[^a-zA-Z]', '', regex=True)
users['location_state'] = users['location_state'].str.replace(r'[^a-zA-Z]', '', regex=True)
users['location_country'] = users['location_country'].str.replace(r'[^a-zA-Z]', '', regex=True)

In [133]:
users[users['location_country'].str.contains('esp')]['location_country']

38517    espaa
44469    espaa
46596    espaa
49959    espaa
50002    espaa
51605    espaa
Name: location_country, dtype: object

In [130]:
# australia
users.loc[users['location_country'].str.contains('australia'), 'location_country'] = 'australia'
# italy
users.loc[users['location_country'].str.contains('ital'), 'location_country'] = 'italy'
# germany
users.loc[users['location_country'].str.contains('deut'), 'location_country'] = 'germany'
users.loc[users['location_country'].str.contains('germ'), 'location_country'] = 'germany'
# united kingdom
users.loc[users['location_country'].str.contains('eng'), 'location_country'] = 'united kingdom'
users.loc[users['location_country'].str.contains('king'), 'location_country'] = 'united kingdom'
# france
users.loc[users['location_country'].str.contains('fran'), 'location_country'] = 'france'
# usa
users.loc[users['location_country'].str.contains('america'), 'location_country'] = 'usa'
users.loc[users['location_country'].str.contains('usa'), 'location_country'] = 'usa'
users.loc[users['location_country'].str.contains('state'), 'location_country'] = 'usa'
users.loc[users['location_country'].str.contains('sate'), 'location_country'] = 'usa'

In [131]:
users['location_country'].value_counts()[:30]

usa               44010
canada             6208
germany            3536
united kingdom     2951
                   2111
australia          1793
spain              1661
france              803
italy               775
switzerland         455
newzealand          453
netherlands         406
portugal            329
austria             251
malaysia            167
singapore           129
sweden              121
ireland             108
belgium             104
brazil              103
finland              90
philippines          81
argentina            77
japan                74
mexico               66
denmark              62
norway               56
israel               48
southafrica          39
romania              39
Name: location_country, dtype: int64

In [6]:
train.head(1)

,user_id,isbn,rating
0,8,0002005018,4


In [7]:
train_df = train.merge(users, on='user_id', how='left').merge(books, on='isbn', how='left')
test_df = test.merge(users, on='user_id', how='left').merge(books, on='isbn', how='left')

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306795 entries, 0 to 306794
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              306795 non-null  int64  
 1   isbn                 306795 non-null  object 
 2   rating               306795 non-null  int64  
 3   age                  214133 non-null  float64
 4   location_city        306795 non-null  object 
 5   location_state       306795 non-null  object 
 6   location_country     306795 non-null  object 
 7   book_title           306795 non-null  object 
 8   book_author          306795 non-null  object 
 9   year_of_publication  306795 non-null  float64
 10  publisher            306795 non-null  object 
 11  img_url              306795 non-null  object 
 12  language             187711 non-null  object 
 13  category             185574 non-null  object 
 14  summary              187711 non-null  object 
 15  img_path         

In [44]:
train_df[train_df['location_city'] == 'london'].head(1)

,user_id,isbn,rating,age,location_city,location_state,location_country,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
175,12863,0452264464,9,29.0,london,england,united kingdom,Beloved (Plume Contemporary Fiction),Toni Morrison,1994.0,Plume,http://images.amazon.com/images/P/0452264464.0...,en,['Fiction'],Staring unflinchingly into the abyss of slaver...,images/0452264464.01.THUMBZZZ.jpg


In [47]:
train_df[train_df['location_country'] == 'canada']['location_state']#.value_counts().head(20)

Series([], Name: location_state, dtype: object)

In [19]:
train_df['location_state'].value_counts().head(20)

 california          28888
 ontario             12979
 texas               12253
 new york            10557
 georgia             10537
 n/a                  9050
 florida              8868
 illinois             8393
 washington           8077
 pennsylvania         8032
 england              7370
 missouri             7326
                      6919
 ohio                 6761
 virginia             6716
 oregon               6207
 new jersey           5969
 michigan             5764
 british columbia     5360
 north carolina       5100
Name: location_state, dtype: int64

In [ ]:
train_df['']

In [ ]:
fig, ax = plt.subplots(3,5,figsize=(20,8))
for country, ax_ in zip(f['location_country'], ax.flatten()):
    users[(users['location_country']==country)]['age'].value_counts().sort_index().plot(ax=ax_, title=country)
plt.xlim(0,100)
plt.tight_layout()
plt.show()